In [ ]:
# install.packages("NeuralNetTools")

In [ ]:
# libraries
library(readxl)
options(warn = 0)
library(tidyverse)
library(readxl)
library(lubridate)
library(zoo)
library(readxl)
library(neuralnet)
library(knitr)
library(Metrics)
library(NeuralNetTools)

In [ ]:
df <- read_excel("data/UOW_load.xlsx")
head(df)

In [ ]:
df$Dates<-as.Date(df$Dates)

In [ ]:
names(df)[2] <- paste("ninth_hour")
names(df)[3] <- paste("tenth_hour")
names(df)[4] <- paste("eleventh_hour")
head(df)

In [ ]:
summary(df[2:4])

In [ ]:
# insert all the inputs to the one data frame
# creating revious time frame values for autoregressive model 
df_full = df %>%
 mutate(
    nine_lag_1 = lag(df$ninth_hour,1),
    ten_lag_1 = lag(df$tenth_hour,1),
    nine_lag_2 = lag(df$ninth_hour,2),
    ten_lag_2 = lag(df$tenth_hour,2),
    eleven_lag_1 = lag(df$eleventh_hour,1),
    eleven_lag_2 = lag(df$eleventh_hour,2),
    eleven_lag_3 = lag(df$eleventh_hour,3),
    eleven_lag_4 = lag(df$eleventh_hour,4),
    #roll mean
    nine_rollmean_4 = rollmean(ninth_hour,4, fill = NA),
    nine_rollmean_7 = rollmean(ninth_hour,7, fill = NA),
    ten_rollmean_4 = rollmean(tenth_hour,4, fill = NA),
    ten_rollmean_7 = rollmean(tenth_hour,7, fill = NA),
    eleven_rollmean_4 = rollmean(eleventh_hour,4, fill = NA),
    eleven_rollmean_7 = rollmean(eleventh_hour,7, fill = NA)) %>%
#Drop null coulumns
drop_na()


In [ ]:
head(df_full)

## Input vercors

In [ ]:
#1st lag
df_full %>%
pivot_longer(cols = c(2, 3, 5, 6, 9), names_to = "kind", values_to = "energy_level") %>%
ggplot(aes(Dates, energy_level, color = kind)) +
geom_line() +
facet_wrap(~kind) + theme(axis.text.x = element_text(angle = 45, vjust =
0.5, hjust = 1
)) +
labs(x = "",
title = "First Set of Input Variables") +
theme(legend.position = "none")


In [ ]:
#2nd lag
df_full %>%
pivot_longer(cols = c(2, 3, 5, 6, 9, 7, 8, 10), names_to = "kind", values_to = "energy_level") %>%
ggplot(aes(Dates, energy_level, color = kind)) +
geom_line() +
facet_wrap(~kind) + theme(axis.text.x = element_text(angle = 45, vjust =
0.5, hjust = 1
)) +
labs(x = "",
title = "Second Set of Input Variables") +
theme(legend.position = "none")

In [ ]:
#3rd lag
df_full %>%
pivot_longer(cols = c(2, 3, 5, 6, 9, 7, 8, 10, 11, 13, 15, 17), names_to = "kind", values_to = "energy_level") %>%
ggplot(aes(Dates, energy_level, color = kind)) +
geom_line() +
facet_wrap(~kind) + theme(axis.text.x = element_text(angle = 45, vjust =
0.5, hjust = 1
)) +
labs(x = "",
title = "Third Set of Input Variables") +
theme(legend.position = "none")

In [ ]:
#4thd lag
df_full %>%
pivot_longer(cols = c(13, 14, 15, 16, 17, 18), names_to = "kind", values_to = "energy_level") %>%
ggplot(aes(Dates, energy_level, color = kind)) +
geom_line() +
facet_wrap(~kind) + theme(axis.text.x = element_text(angle = 45, vjust =
0.5, hjust = 1
)) +
labs(x = "",
title = "Forth Set of Input Variables") +
theme(legend.position = "none")

# Data Normalization

In [ ]:
head(df_full)

In [ ]:
summary(df_full)

##### min-max normalization

In [ ]:
min_max_normalize <- function(x) {
return ((x - min(x)) / (max(x) - min(x))) }


unnormalize_eleventh_hour <- function(x) {
return( (max(df_full$eleventh_hour) - min(df_full$eleventh_hour))*x + min(df_full$eleventh_hour)) }


In [ ]:
df_normalized <- as.data.frame(lapply(df_full[2:18], min_max_normalize))
head(df_normalized)

In [ ]:
summary(df_normalized)

In [ ]:
dim(df_normalized)

In [ ]:
set.seed(123)
df_train <- df_normalized[1:430,]
df_test <- df_normalized[431:493,]

In [ ]:
model_one_hidden_layer = function(arg, hidden_first, activation_fun, learning_rate, test_data, model_type) {
    one_layer_nurelnet = neuralnet(arg,
    data=df_train, 
    hidden = c(hidden_first), 
    linear.output = TRUE, 
    act.fct = activation_fun,
    learningrate = learning_rate,
    algorithm = "rprop+"
    )
    results = compute(one_layer_nurelnet, test_data)
    truth_column = df_full[431:493, 4]$eleventh_hour
    predicted_column =  unnormalize_eleventh_hour(results$net.result)[,1]
      
    results <- data.frame(
        model_type = model_type, 
        hidden_layers = hidden_first, 
        RMSE = rmse(truth_column,predicted_column), 
        MAE = mae(truth_column, predicted_column), 
        MAPE =mape(truth_column, predicted_column),
        act_function = activation_fun,
        learning_rate = learning_rate)
}

In [ ]:
model_two_hidden_layer = function(arg, hidden_first, hidden_second, activation_fun, learning_rate, test_data, model_type) {
    two_layer_nurelnet = neuralnet(arg,
    data=df_train, 
    hidden = c(hidden_first, hidden_second), 
    linear.output = TRUE, 
    act.fct = activation_fun,
    learningrate = learning_rate,
    algorithm = "rprop+"
    )
    results = compute(two_layer_nurelnet, test_data)
    truth_column = df_full[431:493, 4]$eleventh_hour
    predicted_column =  unnormalize_eleventh_hour(results$net.result)[,1]

    results <- data.frame(
        model_type = model_type, 
        hidden_layers = paste0(hidden_first," and ",hidden_second), 
        RMSE = rmse(truth_column,predicted_column), 
        MAE = mae(truth_column, predicted_column), 
        MAPE =mape(truth_column, predicted_column),
        act_function = activation_fun,
        learning_rate = learning_rate)    
}

# Single Layer Neural Networks with different input sets and neurons

1st lag

In [ ]:
set.seed(12345)

In [ ]:
one_layer_neuralnet_first_inset_result = lapply(1:10, function(n){ 
    model_one_hidden_layer(
        "eleventh_hour~ninth_hour+tenth_hour+nine_lag_1+ten_lag_1+eleven_lag_1", 
        n, 
        "logistic", 
        0.01,
        df_test[, c(1, 2, 4, 5, 8)],
        "First Set of Input Variables"
                        )
    })
kable(one_layer_neuralnet_first_inset_result[])

2nd lag

In [ ]:
set.seed(12345)

In [ ]:
one_layer_neuralnet_second_inset_result = lapply(1:10, function(n){ 
    model_one_hidden_layer(
        "eleventh_hour~ninth_hour+tenth_hour+nine_lag_1+ten_lag_1+eleven_lag_1+nine_lag_2+ten_lag_2+eleven_lag_2", 
        n, 
        "logistic", 
        0.01,
        df_test[, c(1, 2, 4, 5, 8, 6, 7, 9)],
        "Second Set of Input Variables"
                        )
    })
kable(one_layer_neuralnet_second_inset_result[])

3rd lag

In [ ]:
set.seed(12345)

In [ ]:
one_layer_neuralnet_third_inset_result = lapply(1:10, function(n){ 
    model_one_hidden_layer(
        "eleventh_hour~ninth_hour+tenth_hour+nine_lag_1+ten_lag_1+eleven_lag_1+nine_lag_2+ten_lag_2+eleven_lag_2+eleven_lag_3+nine_rollmean_4+ten_rollmean_4+eleven_rollmean_4", 
        n, 
        "logistic", 
        0.01,
        df_test[, c(1, 2, 4, 5, 8, 6, 7, 9, 10, 12, 14, 16)],
        "Third Set of Input Variables"
                        )
    })
kable(one_layer_neuralnet_third_inset_result[])

4th lag

In [ ]:
set.seed(12345)

In [ ]:
one_layer_neuralnet_forth_inset_result = lapply(1:10, function(n){ 
    model_one_hidden_layer(
        "eleventh_hour~ninth_hour+tenth_hour+nine_rollmean_4+ten_rollmean_4+eleven_rollmean_4+nine_rollmean_7+ten_rollmean_7+eleven_rollmean_7", 
        n, 
        "logistic", 
        0.01,
        df_test[, c(1, 2, 12, 14, 16, 13, 15, 17)],
        "Forth Set of Input Variables"
                        )
    })
kable(one_layer_neuralnet_forth_inset_result[])

##### minimum error in each input set

In [ ]:
kable(one_layer_neuralnet_first_inset_result[8])

In [ ]:
kable(one_layer_neuralnet_second_inset_result[4])

In [ ]:
kable(one_layer_neuralnet_third_inset_result[5])

In [ ]:
kable(one_layer_neuralnet_forth_inset_result[3])

among above network structures network with 5 nodes in hidden layer and 3rd input set shows the minimum error

In [ ]:
kable(one_layer_neuralnet_third_inset_result[5])

# Two hidden layer Neural Networks with different input sets and neurons

1st lag

In [ ]:
set.seed(12345)

In [ ]:
two_layer_neuralnet_first_inset_result = lapply(1:10, function(n){ 
    lapply(1:10, function(m){
    model_two_hidden_layer(
        "eleventh_hour~ninth_hour+tenth_hour+nine_lag_1+ten_lag_1+eleven_lag_1", 
        n,
        m, 
        "logistic", 
        0.01,
        df_test[, c(1, 2, 4, 5, 8)],
        "First Set of Input Variables with 2 hidden layers"
                        )
    })})
kable(two_layer_neuralnet_first_inset_result[])

2nd lag

In [ ]:
set.seed(12345)

In [ ]:
two_layer_neuralnet_second_inset_result = lapply(1:10, function(n){
    lapply(1:10, function(m){ 
    model_two_hidden_layer(
        "eleventh_hour~ninth_hour+tenth_hour+nine_lag_1+ten_lag_1+eleven_lag_1+nine_lag_2+ten_lag_2+eleven_lag_2", 
        n,
        m, 
        "logistic", 
        0.01,
        df_test[, c(1, 2, 4, 5, 8, 6, 7, 9)],
        "Second Set of Input Variables with 2 hidden layers"
                        )
    })})
kable(two_layer_neuralnet_second_inset_result[])

3d lag

In [ ]:
set.seed(12345)

In [ ]:
two_layer_neuralnet_third_inset_result = lapply(1:10, function(n){
    lapply(1:10, function(m){  
    model_two_hidden_layer(
        "eleventh_hour~ninth_hour+tenth_hour+nine_lag_1+ten_lag_1+eleven_lag_1+nine_lag_2+ten_lag_2+eleven_lag_2+eleven_lag_3+nine_rollmean_4+ten_rollmean_4+eleven_rollmean_4", 
        n,
        m, 
        "logistic", 
        0.01,
        df_test[, c(1, 2, 4, 5, 8, 6, 7, 9, 10, 12, 14, 16)],
        "Third Set of Input Variables with 2 hidden layers"
                        )
    })})
kable(two_layer_neuralnet_third_inset_result[])

4th lag

In [ ]:
set.seed(12345)

In [ ]:
two_layer_neuralnet_forth_inset_result = lapply(1:10, function(n){ 
 lapply(1:10, function(m){  
    model_two_hidden_layer(
        "eleventh_hour~ninth_hour+tenth_hour+nine_rollmean_4+ten_rollmean_4+eleven_rollmean_4+nine_rollmean_7+ten_rollmean_7+eleven_rollmean_7", 
        n,
        m, 
        "logistic", 
        0.01,
        df_test[, c(1, 2, 12, 14, 16, 13, 15, 17)],
        "Forth Set of Input Variables with 2 hidden layers"
                        )
    })})
kable(two_layer_neuralnet_forth_inset_result[])

minimum error in each input set

In [ ]:
kable(two_layer_neuralnet_first_inset_result[8])
# 8 and 2

In [ ]:
kable(two_layer_neuralnet_second_inset_result[4])
# 4 and 6

In [ ]:
kable(two_layer_neuralnet_third_inset_result[10])
# 10 and 3

In [ ]:
kable(two_layer_neuralnet_forth_inset_result[3])
# 3 and 7

best one from layer 2 approach is 3 and 10 nodes with input set 3

## act function and learning rate

layer 1 : input set 3 and 5 nodes

layer 2 : input set 3 and 10 and 3 nodes

minimum error gives the layer 2 approach and going forward with it

changing learning rate

In [ ]:
l_rate=0.001
set.seed(12345)
for (i in 1:100) {
two_layer_neuralnet_third_inset_result = model_two_hidden_layer(
        "eleventh_hour~ninth_hour+tenth_hour+nine_lag_1+ten_lag_1+eleven_lag_1+nine_lag_2+ten_lag_2+eleven_lag_2+eleven_lag_3+nine_rollmean_4+ten_rollmean_4+eleven_rollmean_4", 
        10,
        3, 
        "logistic", 
        l_rate,
        df_test[, c(1, 2, 4, 5, 8, 6, 7, 9, 10, 12, 14, 16)],
        "Third Set of Input Variables with 2 hidden layers"
                        )

    print(kable(two_layer_neuralnet_third_inset_result[]))
    l_rate=l_rate+0.001
}

In [ ]:
l_rate=0.001
set.seed(12345)
for (i in 1:100) {
two_layer_neuralnet_third_inset_result = model_one_hidden_layer(
        "eleventh_hour~ninth_hour+tenth_hour+nine_lag_1+ten_lag_1+eleven_lag_1+nine_lag_2+ten_lag_2+eleven_lag_2+eleven_lag_3+nine_rollmean_4+ten_rollmean_4+eleven_rollmean_4", 
        5, 
        "logistic", 
        l_rate,
        df_test[, c(1, 2, 4, 5, 8, 6, 7, 9, 10, 12, 14, 16)],
        "Third Set of Input Variables with 2 hidden layers"
                        )

    print(kable(two_layer_neuralnet_third_inset_result[]))
    l_rate=l_rate+0.001
}

In [ ]:


final_two_layer = model_two_hidden_layer(
        "eleventh_hour~ninth_hour+tenth_hour+nine_lag_1+ten_lag_1+eleven_lag_1+nine_lag_2+ten_lag_2+eleven_lag_2+eleven_lag_3+nine_rollmean_4+ten_rollmean_4+eleven_rollmean_4", 
        10,
        3, 
        "logistic", 
        0.037,
        df_test[, c(1, 2, 4, 5, 8, 6, 7, 9, 10, 12, 14, 16)],
        "Third Set of Input Variables with 2 hidden layers"
                        )

    print(kable(final_two_layer[]))

In [ ]:
final_single_layer = model_one_hidden_layer(
        "eleventh_hour~ninth_hour+tenth_hour+nine_lag_1+ten_lag_1+eleven_lag_1+nine_lag_2+ten_lag_2+eleven_lag_2+eleven_lag_3+nine_rollmean_4+ten_rollmean_4+eleven_rollmean_4", 
        5, 
        "logistic", 
        0.087,
        df_test[, c(1, 2, 4, 5, 8, 6, 7, 9, 10, 12, 14, 16)],
        "Third Set of Input Variables with 2 hidden layers"
                        )

    print(kable(final_single_layer[]))

predictions

In [ ]:
best_nn = neuralnet(
    "eleventh_hour~ninth_hour+tenth_hour+nine_lag_1+ten_lag_1+eleven_lag_1+nine_lag_2+ten_lag_2+eleven_lag_2+eleven_lag_3+nine_rollmean_4+ten_rollmean_4+eleven_rollmean_4",
    data=df_train, 
    hidden = c(10, 3), 
    linear.output = TRUE, 
    act.fct = "logistic",
    learningrate = 0.037,
    algorithm = "rprop+"
    )
results = compute(best_nn, df_test[, c(1, 2, 4, 5, 8, 6, 7, 9, 10, 12, 14, 16)])
truth_column = df_full[431:493, 4]$eleventh_hour
predicted_column =  unnormalize_eleventh_hour(results$net.result)[,1]
results_df <- data.frame(
    date = df_full[431:493,]$Dates,
    actual = truth_column,
    predicted = predicted_column)

In [ ]:
plotnet(best_nn)

In [ ]:
rmse(truth_column,predicted_column)

In [ ]:
results_df

In [ ]:
summary(results_df)

In [ ]:
# p = ggplot() + 
#   geom_line(data = results_df, aes(x = date, y = actual), color = "blue") +
#   geom_line(data = results_df, aes(x = date, y = predicted), color = "red") +
#   scale_color_manual(name = "energy level", values = c("actual" = "blue", "predicted" = "red"))+
#   xlab('Dates') +
#   ylab('eleventh hour energy level')

# print(p)

In [ ]:
ggplot() +
  geom_line(data = results_df, aes(y = actual, x = date, colour = "actual")) +
  geom_line(data = results_df, aes(y = predicted, x = date, colour = "predicted")) +
  scale_color_manual(name = "energy level", values = c("actual" = "blue", "predicted" = "red")) +
  xlab('Dates') +
  ylab('eleventh hour energy level')